In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if ollama_api_key:
    print(f"OLLAMA API Key exists and begins {ollama_api_key[:2]}")
else:
    print("OLLAMA API Key not set (and this is optional)")

ollama_url = "http://localhost:11434/v1"

openai = OpenAI()
ollama = OpenAI(api_key=ollama_api_key, base_url=ollama_url)


In [38]:
system_prompt_doc = """You are an expert Python developer and code reviewer.
Your job is to read the user's provided function, and return:
1. A concise, PEP-257-compliant docstring summarizing what the function does, clarifying types, parameters, return values, and side effects.
2. Helpful inline comments that improve both readability and maintainability, without restating what the code obviously does.

Only output the function, not explanations or additional text. 
Do not modify variable names or refactor the function logic.
Your response should improve the code's clarity and documentation, making it easier for others to understand and maintain.
Don't be extremely verbose.
Your answer should be at a senior level of expertise.
"""

system_prompt_tests = """You are a seasoned Python developer and testing expert.
Your task is to read the user's provided function, and generate:
1. A concise set of meaningful unit tests that thoroughly validate the function's correctness, including typical, edge, and error cases.
2. The tests should be written for pytest (or unittest if pytest is not appropriate), use clear, descriptive names, and avoid unnecessary complexity.
3. If dependencies or mocking are needed, include minimal necessary setup code (but avoid over-mocking).

Only output the relevant test code, not explanations or extra text.
Do not change the original function; focus solely on comprehensive, maintainable test coverage that other developers can easily understand and extend.
"""

In [39]:
models = ["gpt-4.1-mini", "llama3.1"]
clients = {"gpt-4.1-mini": openai, "llama3.1": ollama}

def generate_documentation(code, model):
    response = clients[model].chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_doc},
            {"role": "user", "content": code}
        ],
        stream=True
    )
    output = ""
    for chunk in response:
        output += chunk.choices[0].delta.content or ""
        yield output.replace("```python", "").replace("```", "")

def generate_tests(code, model):
    response = clients[model].chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_tests},
            {"role": "user", "content": code}
        ],
        stream=True
    )
    output = ""
    for chunk in response:
        output += chunk.choices[0].delta.content or ""
        yield output.replace("```python", "").replace("```", "")

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(spacing_size=gr.themes.sizes.spacing_sm, radius_size=gr.themes.sizes.radius_none)) as ui:
    gr.Markdown("# Python Toolbox", elem_id="app-title")
    
    with gr.Tab("Docstring Generator") as tab1:
        gr.Markdown("## Docstring & Comment Generator")
        gr.Markdown("Paste your function below to generate helpful docstrings and inline comments!")

        with gr.Row():
            with gr.Column():
                code_input = gr.Code(label="Your Python function here", lines=20, language="python")
                model_dropdown = gr.Dropdown(choices=models, value=models[0], label="Select model")
                submit_doc_btn = gr.Button("Generate docstring & comments")
            with gr.Column():
                code_output = gr.Code(label="New function with docstring and comments", language="python")

        submit_doc_btn.click(
            generate_documentation, 
            inputs=[code_input, model_dropdown], 
            outputs=code_output
        )

    with gr.Tab("Unit Tests Generator") as tab2:
        gr.Markdown("## Unit Test Generator")
        gr.Markdown("Paste your function below to generate helpful unit tests!")

        with gr.Row():
            with gr.Column():
                code_input_2 = gr.Code(label="Your Python function here", lines=20, language="python")
                model_dropdown_2 = gr.Dropdown(choices=models, value=models[0], label="Select model")
                submit_test_btn = gr.Button("Generate unit tests")
            with gr.Column():
                code_output_2 = gr.Code(label="Generated unit tests", language="python")

        submit_test_btn.click(
            generate_tests, 
            inputs=[code_input_2, model_dropdown_2], 
            outputs=code_output_2
        )
    
    
    tab1.select(lambda x: x, inputs=code_input_2, outputs=code_input)
    tab2.select(lambda x: x, inputs=code_input, outputs=code_input_2)

ui.launch(share=False, inbrowser=True)